This python script pulls geospatial data and saves it as rasters.
This script shows you how to use Google Earth Engine in python.
There is another option: using rgee in R. However, this is just a wrapper
for the python version of ee, meaning you still need python installed.
Importantly, getting R to work well with python can be difficult and time consuming.
As such, we will just learn how to do this in python, using the Google Colab. One nice thing about Google Colab is that we do not need to install Python or packages! This code should work out of the box:

In [7]:
# we will use geopandas to load shapefiles
import geopandas

# import ee
import ee
# authenticate
ee.Authenticate()
# note that you need to use YOUR PROJECT NAME here:
ee.Initialize(project="ee-geefolder")

Now we need to load the malawi shapefile. You can do this by adding the shapefile to your Google Drive and then mounting the drive on the left-hand side of this page. You can find detailed instructions in the how-to guide, but here is the code:

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now we can load the shapefile. This code will work as long as you put the shapefile directly into Google Drive, without any other folders in your Drive:

In [9]:

# the shapefile:
shape = geopandas.read_file("/content/drive/MyDrive/mw3.shp")
# make sure it is in lat/lon (project it)
shape = shape.to_crs("EPSG:4326")
# let's get the total bounds for the shapefile
bounds = shape.total_bounds

The above code loads the shapefile as an object called `shape`, then reprojects it into longitude/latitude, and finally gets the "bounds" of the shape, which we will use below.

Next, we want to load some type of raster from Google Earth Engine. Let's start with NDVI. This code will get information about NDVI from GEE, print some information (just to make sure we successfully found the NDVI rasters), filter the NDVI rasters by date (in this case, only January of 2019), and then create a single image that we can download:

In [10]:
# Let's look at NDVI
ndvi = ee.ImageCollection("MODIS/061/MOD13A3")

# we can use "print" and "getInfo()" to look at more information
# If the above call worked correctly, you should see a bunch of information printed in the console
print(ndvi.getInfo())

# We can also filter the collection by date. Let's look only at january 2019 (the same year as the IHS5)
ndvi = ndvi.filterDate("2019-01-01", "2019-01-31")

# for assets that have many bands (raster layers), we can select the specific ones we want:
ndvi = ndvi.select("NDVI")
# finally, just make sure we have an IMAGE, not an image collection
ndvi = ndvi.mean()

{'type': 'ImageCollection', 'bands': [], 'version': 1733337580138646, 'id': 'MODIS/061/MOD13A3', 'features': [{'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [43200, 18000], 'crs': 'SR-ORG:6974', 'crs_transform': [926.625433055833, 0, -20015109.354, 0, -926.6254330558334, 10007554.677003]}, {'id': 'EVI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [43200, 18000], 'crs': 'SR-ORG:6974', 'crs_transform': [926.625433055833, 0, -20015109.354, 0, -926.6254330558334, 10007554.677003]}, {'id': 'DetailedQA', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [43200, 18000], 'crs': 'SR-ORG:6974', 'crs_transform': [926.625433055833, 0, -20015109.354, 0, -926.6254330558334, 10007554.677003]}, {'id': 'sur_refl_b01', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimension

Now, we are going to create a "bounding box" (so that GEE will only output a raster in the geographic area we're interested in), create a "task" using the GEE API, and then START the task so that GEE will start downloading the raster.

In [11]:
# let's create a bounding box in earth engine. Note the syntax (xmin, ymin, xmax, ymax)
# this does not accept an array (which is what bounds was), so we will extract the individual components
# Also note that indexing in python starts at 0, not 1! bounds[0] gives the first value in the array
bbox = ee.Geometry.BBox(bounds[0], bounds[1], bounds[2], bounds[3])

# let's set up the task:
task = ee.batch.Export.image.toDrive(image=ndvi,
    description='ndvi1',
    scale=1000, # set scale the same as the raster's resolution (you can find this on GEE)
    region=bbox,
    crs='EPSG:4326',
    fileFormat='GeoTIFF')
# start the task. You MUST do this for GEE to actually run the command.
task.start()
# can check the status of the task
task.status()
# When it is done, you will find the resulting .tiff file in your GOOGLE DRIVE

{'state': 'READY',
 'description': 'ndvi1',
 'priority': 100,
 'creation_timestamp_ms': 1734396585909,
 'update_timestamp_ms': 1734396585909,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'B4CHRH5RGSRLV5CMUESPYTD4',
 'name': 'projects/ee-geefolder/operations/B4CHRH5RGSRLV5CMUESPYTD4'}

As mentioned in the code, when this finishes running, the resulting raster will show up in your Google Drive.

As a final example, we don't usually want just one month of NDVI data. What if we want to download monthly NDVI for the entire year of 2019? This code will allow you to do that:

In [12]:
# What if we want to download all 12 months of 2019? We can set up a loop to do this.
# we will need the package calendar to do this
import calendar
# load the collection
ndvi = ee.ImageCollection("MODIS/061/MOD13A3")
# just keep NDVI
ndvi = ndvi.select("NDVI")


# We can also filter the collection by date. Let's look only at january 2019 (the same year as the IHS5)
# the range function creates numbers from 1 to the second number MINUS 1 (so this is 1 through 12)
for m in range(1, 13):
    # first day of the month is always the first
    firstday = "01"
    # we can use calendar.monthrange to find the LAST day of the month (which is different in each month)
    lastday = calendar.monthrange(2019, m)[1]
    # create dates now
    firstday = "2019-" + str(m) + "-" + firstday
    lastday = "2019-" + str(m) + "-" + str(lastday)
    # finally, filter the dataset (do'nt overwrite it! create new object)
    ndvimonth = ndvi.filterDate(firstday, lastday)
    # and take the mean
    ndvimonth = ndvimonth.mean()
    # set up the export task
    task = ee.batch.Export.image.toDrive(image=ndvimonth,
        description="ndvi" + str(m),
        scale=1000,
        region=bbox,
        crs='EPSG:4326',
        fileFormat='GeoTIFF')
    # start the task.
    task.start()

It will take a little bit of time, but if you are patient, you will eventually have 12 .tiff files in your Google Drive, each with a different name: "ndviM", where "M" is from 1 to 12 (for each month).